# Update DSS User Preferences
This sample is for the DSS(DataScope Select) users who would like to change their user preferences using DSS REST API in Python.


## Prerequisites
- Python 3.6 or higher
- Jupyter Notebook
- DSS username and password.

## Implementation
### Step 1. Request authentication token

 - Import required modules

In [ ]:
import getpass as gp
import requests
import json
import pandas as pd
import copy

 - Input DSS username and password

In [ ]:
username=input('Enter DSS username:')
password=gp.getpass('Enter DSS Password:')

 - Create authentication token request containing DSS username and password

In [ ]:
requestUrl = "https://hosted.datascopeapi.reuters.com/RestApi/v1/Authentication/RequestToken"
requestHeaders={
    "Prefer":"respond-async",
    "Content-Type":"application/json"
    }
requestBody={
    "Credentials": {
    "Username": username,
    "Password": password
  }
}

- send the request and waits for the response

In [ ]:
authenticationResp = requests.post(requestUrl, json=requestBody,headers=requestHeaders)
print("Received the response for authentication request")

- Check if the status code of the response is 200. If yes, the request has succeeded so extracts and prints the authentication token. Otherwise, print the error.

In [ ]:
if authenticationResp.status_code == 200 :
    print("Received status code 200, get the authentication token from the response")
    jsonResponse = json.loads(authenticationResp.text.encode('ascii', 'ignore'))
    token = jsonResponse["value"]
    print ('Authentication token (valid 24 hours):')
    print (token)
else:
    print("Error with status code:",authenticationResp.status_code,"\n Text:",json.dumps(json.loads(authenticationResp.text),indent=4))

### Step 2. Get the Current User Preferences


- create user preferences request

In [ ]:
requestUrl='https://hosted.datascopeapi.reuters.com/RestApi/v1/Users/UserPreferences'
requestHeaders={
    "Prefer":"respond-async",
    "Authorization": "token " + token
}

- send the request and waits for the response

In [ ]:
userPreferencesResp = requests.get(requestUrl, headers=requestHeaders)
print("Received the response for user preferences request")

- Check if the status code of the response is 200. If yes, the request has succeeded so extracts and prints the user preferences. Otherwise, print the error.

In [ ]:
userPreferencesJson = None
if userPreferencesResp.status_code == 200 :
    print("Received status code 200, requested for user preferences successfully")
    userPreferencesJson = json.loads(userPreferencesResp.text.encode('ascii', 'ignore'))
    wholePref = userPreferencesJson["value"][0]
else:
    print("Error with status code:",userPreferencesResp.status_code,"\n Text:",json.dumps(json.loads(userPreferencesResp.text),indent=4))

- Function to get and display user preferences  

In [ ]:
def getUserPreference(preference, paramsName):
    dfSetting = pd.DataFrame(list(preference.items()), columns=[paramsName, 'Value'])
    dfSetting.index += 1
    print("\t " + paramsName +" table")
    return dfSetting

- Get and display Content Setting user preferences  

In [ ]:
contentSetting = wholePref['ContentSettings']
dfContent = getUserPreference(contentSetting, "Content Settings Parameter")  
dfContent

- Get and display UI setting user preferences  

In [ ]:
uiSetting = wholePref['UiSettings']
dfUISetting = getUserPreference(uiSetting, "UI Settings Parameter")  
dfUISetting

### Step 3. Update User Preferences


- Function to input the user preferences parameters which will be updated.

In [ ]:
def getUpdatedUserPreference(prefParams, prefType, example):
    updatedParams = {}
    print("Please see " + prefType + " Table above.")
    updatedInput=input("Enter an updated " + prefType + " e.g." + example + " (press enter to exit):")
    while len(updatedInput) > 0:
        updatedIndexValue=updatedInput.split(" ",1)
        updatedIndex = int(updatedIndexValue[0])
        index = 0
        for key in prefParams:
            index += 1
            if index==updatedIndex:
                prefParams[key] = updatedIndexValue[1]
                print(key + " will be updated to " + prefParams[key])
                print()
        updatedInput=input("Enter an updated "+ prefType + " (press enter to exit):")
    updatedParams = copy.deepcopy(prefParams)  
    return updatedParams

- Select the updated Content Setting user preferences and their new values. 

- The valid values of **PreferredIdentifier** are **RIC**, **CSP**, **ISN**, **VAL**, **WPK**, **COM**, **SED**. CSP is Cusip, ISN is ISIN, VAL is Valoren, WPK is Wertpapier, COM is Common Code and SED is Sedol.

- For the valid values of another parameters, please refer to [Update User Preference in REST API Reference Tree](https://hosted.datascopeapi.reuters.com/RestApi.Help/Context/Operation?ctx=Users&ent=UserPreference&opn=Update)



In [ ]:
updatedContentParams=getUpdatedUserPreference(contentSetting,"Content Settings Parameter", "2 False")

- Select the updated UI Setting user preferences and their new values. For the valid values of each parameters, please refer to [Update User Preference in REST API Reference Tree](https://hosted.datascopeapi.reuters.com/RestApi.Help/Context/Operation?ctx=Users&ent=UserPreference&opn=Update)

In [ ]:
updatedUIParams=getUpdatedUserPreference(uiSetting,"UI Settings Parameter", "2 hh:mm tt")

- Create a Request for update user preferences

In [ ]:
requestUrl='https://hosted.datascopeapi.reuters.com/RestApi/v1/Users/UserPreferences('+username+')'
requestHeaders={
    "Prefer":"respond-async",
    "Authorization": "token " + token
}
requestBody={
    "UserPreferenceId": int(username),
    "ContentSettings": 
       updatedContentParams,
     "UiSettings": 
       updatedUIParams
}

- Send HTTP Put request for update user preferences

In [ ]:
updatedPrefResp = requests.put(requestUrl, json=requestBody,headers=requestHeaders)
print("Received the response for update user preferences request")

- Check if user preferences has been updated successfully

In [ ]:
if updatedPrefResp.status_code == 204 :
    print("Received status code 204, updated user preferences successfully")
else:
    print("Error with status code:",updatedPrefResp.status_code,"\n Text:",json.dumps(json.loads(updatedPrefResp.text),indent=4))